    # Clase - Computación Distribuida

    ## Pyspark Hands-on 
    #### Marcelo Medel Vergara - Diplomado Data Engineer USACH

## Instalación de librerías necesarias

Si utilizas Anaconda para administrar ambientes de desarrollo la mejor vía para asegurar que funcionen correctamente las librerías es instalando directo desde Conda. Adicionalmente se instalan todas las librerías que necesita Spark.
- `conda create -n pyspark-DE`
- `conda activate pyspark-DE`
- `conda install -c conda-forge pyspark python=3.10`

De igual forma se puede instalar a través de PIP, pero sin asegurar funcionamiento correcto ni la instalación de dependencias. Adicionalmente será necesario tener instalado Pandas.
- `conda create -n pyspark-DE python=3.10`
- `conda activate pyspark-DE`
- `pip install pyspark`
- `pip install pandas`.

Para Google Colab:
- `!pip install pyspark`

## SparkSession

SparkSession es una clase en PySpark que existe desde la versión 2.0 (2016) que simplifica la forma de trabajar con Spark, tanto en las configuraciones como en la manipulación de datos estructurados. 


#### Funcionalidades principales de SparkSession:

1. **Configura Spark**: Para profundizar en las configuraciones posibles de Spark, visite https://spark.apache.org/docs/latest/configuration.html.
    - `SparkSession.builder.appName("some name").**config("some config key,value")**.getOrCreate()`
    - `spark.conf.get("some config key")`

2. **Crear DataFrames**: permite leer y escribir (Input/Output) diversas fuentes de datos y crear DataFrames para la manipulación de datos.
    - `spark.createDataFrame(data [, schema])`
    - `spark.read.json("path to some json")`

3. **Ejecutar SQL**: facilita la ejecución de consultas en SQL sobre los DataFrames.
    - `spark.sql("query to some view")`
    
4. **Gestiona contexto de Spark**: facilita la configuración y acceso a diferentes componentes y funcionalidades de Spark
    - `spark.sql.shuffle.partitions`
    - `spark.executor.memory`
    - `spark.catalog.listTables()`
    - `spark.catalog.listColumns("someTable")`
    - `spark.udf.register("someName", someUdf)`

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .appName("hands-on-pyspark")\
        .config("spark.executor.memory","2g")\
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR") #ALL, OFF, ERROR, DEBUG, INFO, WARN
spark.active()

## DataFrame en Spark

Un DataFrame es una estructura de datos bidimensional similar a cualquier tabla en una base de datos estructurada. Algunas de las características que tiene un DataFrame en PySpark son:
- **Distribuido**: Los datos están distribuidos en un clúster de nodos, lo que permite el procesamiento paralelo.

- **Inmutable**: Cada transformación produce un nuevo DataFrame.

- **SQL**: Permite operaciones tipo SQL y ofrece una interfaz similar a Pandas pero a gran escala.

- **Conexiones diversas**: Puede leer datos de múltiples fuentes (ej: JSON, CSV, Parquet, JDBC).

- **Optimización Automática**: Utiliza *Catalyst Optimizer* para optimizar automáticamente las consultas.

### Creación de un DataFrame

### Data Types

## Operaciones básicas sobre un DataFrame

### Select, Columns y Expressions

***Select*** permite seleccionar un set de columnas y también puede ser usado para renombrar o aplicar *expresiones* a las columnas.

***Columns*** en Spark hace referencia a cualquier columna conocida en algun RDBMS, planilla de excel, pandas dataframes, etc. Estas columnas pueden ser seleccionadas, manipuladas o removidas de un DataFrame, por lo que estas operaciones son representadas como ***expressions***

-  `col` y `column` son utilizadas para referencias a columnas en un DataFrame
- `expr` es utilizado para crear expresiones mas complejas, recibe un string con SQL para ejecutar la operación.


### Literals, withColumn, withColumnRenamed y alias

### Filter, where, drop y cast

### Pandas DataFrame to Spark DataFrame

![spark-pandas](./files/spark_pandas.png)